In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from transformers import get_scheduler
import torch
from torch.utils.data import DataLoader, Dataset
from seqeval.metrics import classification_report
import sqlite3


In [23]:

# Load your dataset
data = pd.read_csv('C:\\Users\\Hasan\\Desktop\\data science folder\\telegram_data.csv', encoding='latin1')

# Check the first few rows to understand its structure
print(data.head())


         Channel Title    Channel Username    ID Message  \
0  Sheger online-store  @Shageronlinestore  5333     NaN   
1  Sheger online-store  @Shageronlinestore  5332     NaN   
2  Sheger online-store  @Shageronlinestore  5331     NaN   
3  Sheger online-store  @Shageronlinestore  5330     NaN   
4  Sheger online-store  @Shageronlinestore  5329     NaN   

                        Date                          Media Path  
0  2024-09-20 11:50:03+00:00  photos/@Shageronlinestore_5333.jpg  
1  2024-09-20 11:50:03+00:00  photos/@Shageronlinestore_5332.jpg  
2  2024-09-20 11:50:03+00:00  photos/@Shageronlinestore_5331.jpg  
3  2024-09-20 11:50:02+00:00  photos/@Shageronlinestore_5330.jpg  
4  2024-09-20 11:50:02+00:00  photos/@Shageronlinestore_5329.jpg  


In [34]:
import re
import pandas as pd
from nltk.corpus import stopwords
import nltk

# Download stopwords if not already done
nltk.download('stopwords')

# Set of stop words (adjust for your language if needed)
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """
    Preprocesses the input text by:
    1. Handling NaN values
    2. Removing URLs, mentions, and non-alphanumeric characters
    3. Converting to lowercase
    4. Removing stopwords (optional)
    """
    # Handle NaN values by returning an empty string
    if pd.isna(text):
        return ''
    
    # Remove URLs, mentions, and non-alphanumeric characters
    text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions (usernames)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase

    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)

# Assuming `data` is a pandas DataFrame with a 'Message' column
# Apply preprocessing only to non-NaN rows
data['processed_message'] = data['Message'].apply(preprocess_text)

# Check the results
print(data[['Message', 'processed_message']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Message processed_message
0     NaN                  
1     NaN                  
2     NaN                  
3     NaN                  
4     NaN                  


In [41]:

# Extract the 'Channel Username' column
channels = data['Channel Username'].dropna().tolist()

# Ensure there are at least 5 channels
if len(channels) < 5:
    print("Not enough channels in the dataset. Please add more.")
else:
    selected_channels = channels[:5]  # Select the first 5 channels
    print(f"Selected Channels: {selected_channels}")



Selected Channels: ['@Shageronlinestore', '@Shageronlinestore', '@Shageronlinestore', '@Shageronlinestore', '@Shageronlinestore']


In [43]:
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already done
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Handle NaN values by returning an empty string
    if pd.isna(text):
        return ''
    
    # Remove URLs, mentions, and non-alphanumeric characters
    text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions (usernames)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase

    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)

# Example usage
text = "Check out our new product, available for 5000 ETB!"
cleaned_text = preprocess_text(text)
print(f"Cleaned Text: {cleaned_text}")


Cleaned Text: check new product available 5000 etb


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
# Example data (you would replace this with your actual fetched data)
data = {
    'channel_name': ['Channel 1', 'Channel 2'],
    'text': ['Product 1 for 1000 ETB', 'Product 2 for 2000 ETB'],
    'image_path': ['path_to_image1', 'path_to_image2']
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('processed_data.csv', index=False)


In [50]:
print(type(data))  # Should output <class 'pandas.core.frame.DataFrame'>


<class 'dict'>
